<a href="https://colab.research.google.com/github/K-Hebel/WiDS_Chi/blob/master/Model_WiDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Libraries

In [1]:
#@title
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

#Load DataFile 'final_sample.csv' - A 50,000 record sample of orginal training dataset (Cleaned Based on EDA comments -df_A)

In [11]:
df_A=pd.read_csv('/content/drive/My Drive/final_sample.csv')

In [12]:
df_A.describe()

,Unnamed: 0,age,bmi,weight,arf_apache,bun_apache,creatinine_apache,glucose_apache,d1_diasbp_invasive_min,d1_bun_max,d1_bun_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hemaglobin_max,d1_potassium_max,h1_bun_max,h1_bun_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_potassium_max,h1_potassium_min,aids,cirrhosis,hepatic_failure,solid_tumor_with_metastasis,diabetes_mellitus
count,50000.00000,48413.000000,48297.000000,48671.000000,50000.000000,40759.000000,40961.000000,45802.000000,13461.000000,45143.000000,45143.000000,45312.000000,45312.000000,47683.000000,47683.000000,43867.000000,45627.000000,10142.000000,10142.000000,10219.000000,10219.000000,23530.000000,23530.000000,11878.000000,11878.000000,50000.000000,50000.000000,50000.00000,50000.00000,50000.000000
mean,64517.25396,0.706565,0.287905,0.325286,0.041300,0.193007,0.120662,0.256355,0.491915,0.193565,0.202304,0.119614,0.122323,0.232773,0.333383,0.436856,0.356713,0.179845,0.179676,0.116601,0.116768,0.202849,0.215413,0.367910,0.367195,0.000780,0.017040,0.01368,0.01968,0.500000
std,37304.10042,0.179790,0.162890,0.175116,0.198985,0.175740,0.152694,0.185362,0.149649,0.175884,0.177891,0.153284,0.150881,0.184414,0.167973,0.204684,0.163933,0.170784,0.170673,0.149090,0.149336,0.170662,0.162622,0.166125,0.168320,0.027918,0.129422,0.11616,0.13890,0.500005
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
25%,31991.25000,0.606742,0.177694,0.200814,0.000000,0.073171,0.040441,0.110774,0.404762,0.073770,0.081751,0.041783,0.045653,0.102230,0.227451,0.278846,0.238095,0.068702,0.068702,0.041688,0.041811,0.092761,0.111465,0.255319,0.260870,0.000000,0.000000,0.00000,0.00000,0.000000
50%,64325.50000,0.730337,0.258489,0.298507,0.000000,0.130081,0.068934,0.219761,0.488095,0.131148,0.145336,0.067781,0.072630,0.182156,0.301961,0.432692,0.333333,0.122137,0.122137,0.066524,0.066720,0.150933,0.167197,0.340426,0.347826,0.000000,0.000000,0.00000,0.00000,0.500000
75%,96233.75000,0.842697,0.363143,0.416554,0.000000,0.252033,0.128676,0.343041,0.571429,0.245902,0.263421,0.126277,0.127621,0.306691,0.403922,0.576923,0.428571,0.229008,0.229008,0.121516,0.121875,0.253127,0.265924,0.446809,0.456522,0.000000,0.000000,0.00000,0.00000,1.000000
max,130155.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000,1.000000


##Proportion of Missing Data in df_A by Feature Column and Count of Available data by Feature Column

In [13]:
missing_data= pd.DataFrame(df_A.isna().sum()/df_A['diabetes_mellitus'].count())
missing_data['Available']= df_A.notna().sum()
missing_data.sort_values(0,ascending=False)

,0,Available
h1_bun_max,0.79716,10142
h1_bun_min,0.79716,10142
h1_creatinine_max,0.79562,10219
h1_creatinine_min,0.79562,10219
h1_potassium_min,0.76244,11878
h1_potassium_max,0.76244,11878
d1_diasbp_invasive_min,0.73078,13461
h1_glucose_max,0.52940,23530
h1_glucose_min,0.52940,23530
bun_apache,0.18482,40759


##Set Features and Target 

In [14]:
y_a=df_A['diabetes_mellitus']
X_a=df_A.drop(['diabetes_mellitus'], axis=1)

##Establish no NaN values in target data

In [15]:
y_a.isna().sum()


0

In [16]:
X_a.isna().sum()

Unnamed: 0                         0
age                             1587
bmi                             1703
weight                          1329
arf_apache                         0
bun_apache                      9241
creatinine_apache               9039
glucose_apache                  4198
d1_diasbp_invasive_min         36539
d1_bun_max                      4857
d1_bun_min                      4857
d1_creatinine_max               4688
d1_creatinine_min               4688
d1_glucose_max                  2317
d1_glucose_min                  2317
d1_hemaglobin_max               6133
d1_potassium_max                4373
h1_bun_max                     39858
h1_bun_min                     39858
h1_creatinine_max              39781
h1_creatinine_min              39781
h1_glucose_max                 26470
h1_glucose_min                 26470
h1_potassium_max               38122
h1_potassium_min               38122
aids                               0
cirrhosis                          0
h

In [18]:
X_a.head()

,Unnamed: 0,age,bmi,weight,arf_apache,bun_apache,creatinine_apache,glucose_apache,d1_diasbp_invasive_min,d1_bun_max,d1_bun_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hemaglobin_max,d1_potassium_max,h1_bun_max,h1_bun_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_potassium_max,h1_potassium_min,aids,cirrhosis,hepatic_failure,solid_tumor_with_metastasis
0,115391,0.539326,0.259137,0.173677,0.0,0.032520,0.033088,0.035733,NaN,0.008197,0.018167,0.029712,0.037353,0.092937,0.101961,0.663462,0.357143,0.007634,0.007634,0.02927,0.029357,0.072322,0.093949,0.382979,0.391304,0,0,0,0
1,69518,0.573034,0.518842,0.450475,0.0,0.121951,0.052390,0.201894,NaN,0.122951,0.118085,0.056639,0.059142,0.146840,0.309804,0.817308,0.452381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,44079,0.865169,0.091209,0.067164,0.0,0.008130,0.009191,0.119707,NaN,0.016393,0.009083,0.005571,0.010376,0.161710,0.282353,0.548077,0.261905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,2516,0.831461,NaN,NaN,0.0,0.105691,0.055147,0.175094,0.369048,0.106557,0.127169,0.051996,0.062254,0.118959,0.407843,0.355769,0.404762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,112777,0.516854,0.227393,0.282225,0.0,0.154472,0.023897,0.096480,NaN,0.155738,0.154419,0.025998,0.026977,0.161710,0.231373,0.125000,0.285714,NaN,NaN,NaN,NaN,0.051883,0.079618,NaN,NaN,0,0,0,0


#Load Original DataFile fo 130,157 records (Clean Based on Prop of Available Data - df_B)

In [19]:
#@title
! pip install -q kaggle
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"khebel","key":"11c0f35aedbce6808a3f45f82709accf"}'}

In [21]:
#@title
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c 'widsdatathon2021'
! unzip TrainingWiDS2021.csv.zip -d train

mkdir: cannot create directory ‘/root/.kaggle’: File exists
DataDictionaryWiDS2021.csv: Skipping, found more recently modified local copy (use --force to force download)
UnlabeledWiDS2021.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
SampleSubmissionWiDS2021.csv: Skipping, found more recently modified local copy (use --force to force download)
TrainingWiDS2021.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
SolutionTemplateWiDS2021.csv: Skipping, found more recently modified local copy (use --force to force download)
Archive:  TrainingWiDS2021.csv.zip
  inflating: train/TrainingWiDS2021.csv  


In [42]:
#@title
df_B=pd.read_csv("train/TrainingWiDS2021.csv")



##Proportion of Missing Data in df_B by Feature Column and Count of Available data by Feature Column

In [43]:
missing_data= pd.DataFrame(df_B.isna().sum()/df_B['diabetes_mellitus'].count())
missing_data['Available']= df_B.notna().sum()
missing_data.sort_values(0,ascending=False)

,0,Available
h1_bilirubin_max,0.920896,10296
h1_bilirubin_min,0.920896,10296
h1_albumin_max,0.914319,11152
h1_albumin_min,0.914319,11152
h1_lactate_min,0.910185,11690
...,...,...
icu_stay_type,0.000000,130157
icu_id,0.000000,130157
elective_surgery,0.000000,130157
hospital_id,0.000000,130157


##Drop Columns with missing data >=75% from df_B

In [44]:
col_drop=missing_data[missing_data[0]>= 0.70].sort_values(by=0, ascending=False)
df_B= df_B.drop(list(col_drop.index), axis=1) #[0].sort_values
df_B.shape

(130157, 126)

##Create Stratified Sample of df_B dataset to balance target variable value counts for sample size of 56,302 (included all positive diabetes_mellitus from original dataset)

In [47]:
df_B['diabetes_mellitus'].value_counts()

0    102006
1     28151
Name: diabetes_mellitus, dtype: int64

In [48]:
sample_df_B=df_B.groupby('diabetes_mellitus', group_keys=False).apply(lambda x: x.sample(28151))
sample_df_B.describe()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,...,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_glucose_max,h1_glucose_min,h1_inr_max,h1_inr_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
count,56302.000000,56302.000000,56302.000000,54456.000000,54415.000000,56302.000000,55514.000000,56302.000000,56302.000000,56302.0,54811.000000,22834.000000,55689.000000,56021.000000,56302.000000,56302.000000,20820.000000,45897.000000,46114.000000,55450.000000,55450.000000,56054.000000,55450.000000,51507.000000,56192.000000,44851.000000,56302.000000,56148.000000,55971.000000,46269.000000,53472.000000,28174.000000,56302.000000,43748.000000,56197.000000,56197.000000,55590.000000,55590.000000,56203.000000,56203.000000,...,49726.000000,49726.000000,51091.000000,51091.000000,53674.000000,53674.000000,48096.000000,48096.000000,49462.000000,49462.000000,49696.000000,49696.000000,21150.000000,21150.000000,48505.000000,48505.000000,51441.000000,51441.000000,51083.000000,51083.000000,48964.000000,48964.000000,26572.000000,26572.000000,21150.000000,21150.000000,20159.000000,20159.000000,20020.00000,20020.000000,20387.000000,20387.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000
mean,64424.550513,212973.132997,106.245231,62.943037,30.068993,0.188572,169.593615,657.825637,0.885478,0.0,86.499782,2.869318,185.476037,562.464101,0.204078,0.040336,1.149357,27.820901,1.611007,3.504382,5.505951,0.010775,4.054734,182.402553,99.683781,32.714352,0.156229,87.297849,25.111608,137.776704,36.416280,1801.914782,0.336187,12.227421,88.325231,49.406463,88.497518,49.462943,103.037525,70.697258,...,8.401655,8.177044,1.626701,1.479934,198.158065,118.152178,24.409660,23.015768,11.339398,10.798219,34.257196,32.685317,1.596001,1.478947,206.694506,195.810425,4.298250,3.945443,139.066474,137.502167,12.573523,11.343461,188.326685,177.554757,1.596001,1.478947,45.280180,38.520225,7.38656,7.320925,163.180031,101.333011,0.000764,0.017903,0.014440,0.025079,0.007797,0.004227,0.019644,0.500000
std,37291.625559,38096.168668,63.857614,16.020970,8.639522,0.391172,10.822080,302.156631,2.633754,0.0,25.908529,0.677663,86.341779,461.950791,0.403030,0.196748,2.250188,21.672706,1.648329,0.924472,1.248249,0.103244,1.523138,103.904120,30.364243,6.730470,0.363075,42.235812,15.039625,5.325971,0.865847,1457.303278,0.472408,6.854296,20.380211,13.287439,20.359052,13.325862,21.770584,17.128556,...,0.736436,0.785279,1.637415,1.446166,99.445274,42.816732,4.462235,5.258082,2.129080,2.306092,6.135988,6.709575,0.970202,0.752923,89.213067,87.495303,0.688436,0.595017,4.835619,4.943650,6.724044,5.866072,108.826843,102.268713,0.970202,0.752923,14.632047,11.166777,0.08663,0.111408,105.851677,59.735820,0.027626,0.132602,0.119297,0.156367,0.087958,0.064880,0.138775,0.500004
min,1.000000,147000.000000,1.000000,0.000000,14.844926,0.000000,137.200000,82.000000,-0.219444,0.0,38.600000,1.200000,101.000000,0.010000,0.000000,0.000000,0.100000,4.0

In [49]:
sample_df_B['diabetes_mellitus'].value_counts()

1    28151
0    28151
Name: diabetes_mellitus, dtype: int64

##Drop Features Not Associated with Patient Vitals

In [50]:
list(sample_df_B.columns)

['Unnamed: 0',
 'encounter_id',
 'hospital_id',
 'age',
 'bmi',
 'elective_surgery',
 'ethnicity',
 'gender',
 'height',
 'hospital_admit_source',
 'icu_admit_source',
 'icu_id',
 'icu_stay_type',
 'icu_type',
 'pre_icu_los_days',
 'readmission_status',
 'weight',
 'albumin_apache',
 'apache_2_diagnosis',
 'apache_3j_diagnosis',
 'apache_post_operative',
 'arf_apache',
 'bilirubin_apache',
 'bun_apache',
 'creatinine_apache',
 'gcs_eyes_apache',
 'gcs_motor_apache',
 'gcs_unable_apache',
 'gcs_verbal_apache',
 'glucose_apache',
 'heart_rate_apache',
 'hematocrit_apache',
 'intubated_apache',
 'map_apache',
 'resprate_apache',
 'sodium_apache',
 'temp_apache',
 'urineoutput_apache',
 'ventilated_apache',
 'wbc_apache',
 'd1_diasbp_max',
 'd1_diasbp_min',
 'd1_diasbp_noninvasive_max',
 'd1_diasbp_noninvasive_min',
 'd1_heartrate_max',
 'd1_heartrate_min',
 'd1_mbp_max',
 'd1_mbp_min',
 'd1_mbp_noninvasive_max',
 'd1_mbp_noninvasive_min',
 'd1_resprate_max',
 'd1_resprate_min',
 'd1_spo2_

In [52]:
drop_cols2 =['Unnamed: 0',
 'encounter_id',
 'hospital_id',
 'elective_surgery',
 'hospital_admit_source',
 'icu_admit_source',
 'icu_id',
 'icu_stay_type',
 'icu_type',
 'pre_icu_los_days',
 'readmission_status',
 ]

sample_df_B2=sample_df_B.drop(drop_cols2, axis=1)
sample_df_B2.shape

(56302, 115)

##Encode Categorical Data in sample_df_B2 (Sample set of df_B)

In [57]:
sample_df_B2.select_dtypes(include=['object']).columns

Index(['ethnicity', 'gender'], dtype='object')

In [60]:
demo_cat=pd.get_dummies(sample_df_B2[['gender','ethnicity']])
sample_df_B3 = sample_df_B2.join(demo_cat)
sample_df_B3=sample_df_B3.drop(['gender','ethnicity'], axis=1)
sample_df_B3.shape

(56302, 121)

##Normalize